# 🌿 PLANT DISEASE CLASSIFICATION USING TENSORFLOW/KERAS 🌿

##### ⚠️⚠️⚠️DISCLAIMER: This notebook is a TensorFlow/Keras implementation of the original PyTorch notebook. It provides the same functionality using TensorFlow and Keras APIs.

# Description of the dataset 📝

This dataset is created using offline augmentation from the original dataset. The original PlantVillage Dataset can be found [here](https://github.com/spMohanty/PlantVillage-Dataset).This dataset consists of about 87K rgb images of healthy and diseased crop leaves which is categorized into 38 different classes. The total dataset is divided into 80/20 ratio of training and validation set preserving the directory structure. A new directory containing 33 test images is created later for prediction purpose.

Note: This description is given in the dataset itself

# Our goal 🎯
Goal is clear and simple. We need to build a model, which can classify between healthy and diseased crop leaves and also if the crop have any disease, predict which disease is it.

##### Let's get started....

## Importing necessary libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from PIL import Image

%matplotlib inline

# 🧭 Exploring the data 🧭

Loading the data 

In [ ]:
data_dir = "../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = data_dir + "/train"
valid_dir = data_dir + "/valid"
diseases = os.listdir(train_dir)

In [ ]:
# printing the disease names
print(diseases)

In [ ]:
print("Total disease classes are: {}".format(len(diseases)))

In [ ]:
plants = []
NumberOfDiseases = 0
for plant in diseases:
    if plant.split('___')[0] not in plants:
        plants.append(plant.split('___')[0])
    if plant.split('___')[1] != 'healthy':
        NumberOfDiseases += 1

In [ ]:
# unique plants in the dataset
print(f"Unique Plants are: \n{plants}")

In [ ]:
# number of unique plants
print("Number of plants: {}".format(len(plants)))

In [ ]:
# number of unique diseases
print("Number of diseases: {}".format(NumberOfDiseases))

In [ ]:
# Number of images for each disease
nums = {}
for disease in diseases:
    nums[disease] = len(os.listdir(train_dir + '/' + disease))
    
# converting the nums dictionary to pandas dataframe passing index as plant name and number of images as column

img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["no. of images"])
img_per_class

In [ ]:
# plotting number of images available for each disease
index = [n for n in range(38)]
plt.figure(figsize=(20, 5))
plt.bar(index, [n for n in nums.values()], width=0.3)
plt.xlabel('Plants/Diseases', fontsize=10)
plt.ylabel('No of images available', fontsize=10)
plt.xticks(index, diseases, fontsize=5, rotation=90)
plt.title('Images per each class of plant disease')

In [ ]:
n_train = 0
for value in nums.values():
    n_train += value
print(f"There are {n_train} images for training")

# 🍳 Data Preparation for training 🍳

## Setting up data generators with augmentation

In [ ]:
# Image parameters
IMG_SIZE = 256
BATCH_SIZE = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation
valid_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

In [ ]:
# Get number of classes
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")

## 👷 Building the model architecture 👷

*We are going to use **ResNet-like architecture**, which have been one of the major breakthrough in computer vision since they were introduced in 2015.*

#### Residual Block implementation in Keras

In [ ]:
def conv_block(x, filters, pool=False):
    """Convolutional block with BatchNormalization"""
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    if pool:
        x = layers.MaxPooling2D(pool_size=4)(x)
    return x

def residual_block(x, filters):
    """Residual block with skip connection"""
    res = x
    x = conv_block(x, filters)
    x = conv_block(x, filters)
    x = layers.Add()([x, res])
    return x

## 👷 Defining the final architecture of our model 👷

In [ ]:
def create_resnet9(input_shape=(256, 256, 3), num_classes=38):
    """Create ResNet9 architecture using Keras Functional API"""
    inputs = layers.Input(shape=input_shape)
    
    # Initial convolution
    x = conv_block(inputs, 64)
    x = conv_block(x, 128, pool=True)  # 128 x 64 x 64
    
    # First residual block
    x = residual_block(x, 128)
    
    # More convolutions
    x = conv_block(x, 256, pool=True)  # 256 x 16 x 16
    x = conv_block(x, 512, pool=True)  # 512 x 4 x 4
    
    # Second residual block
    x = residual_block(x, 512)
    
    # Classifier
    x = layers.GlobalMaxPooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs, name='ResNet9')
    return model

In [ ]:
# Create the model
model = create_resnet9(num_classes=num_classes)
model.summary()

# 🏋️ Training the model 🏋️

## Setting up callbacks and optimizer

In [ ]:
# Hyperparameters
EPOCHS = 10
INITIAL_LR = 0.01

# Callbacks
checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=1e-6,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

callbacks = [checkpoint, reduce_lr, early_stop]

In [ ]:
# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator,
    callbacks=callbacks,
    verbose=1
)

# 📊 Visualizing Training Results 📊

In [ ]:
# Plot training history
def plot_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Train Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Train Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

plot_history(history)

# 🎯 Model Evaluation 🎯

In [ ]:
# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(valid_generator)
print(f"\nValidation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

# 💾 Saving the Model 💾

In [ ]:
# Save the final model
model.save('plant_disease_model_final.keras')
print("Model saved successfully!")

# 🔮 Making Predictions 🔮

In [ ]:
def predict_disease(image_path, model, class_names):
    """Predict disease from an image"""
    img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = np.max(predictions[0])
    
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predicted: {predicted_class}\nConfidence: {confidence:.2%}")
    plt.show()
    
    return predicted_class, confidence

# Get class names
class_names = list(train_generator.class_indices.keys())

## Conclusion

This notebook demonstrates how to build a plant disease classification model using TensorFlow/Keras with a ResNet-inspired architecture. The model achieves high accuracy in classifying 38 different plant disease classes.

Key features:
- ResNet-like architecture with residual connections
- Data augmentation for better generalization
- Learning rate scheduling and early stopping
- Model checkpointing to save the best model

The TensorFlow/Keras implementation provides similar functionality to the PyTorch version while leveraging the high-level Keras API for easier model building and training.